In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from mlxtend.evaluate import PredefinedHoldoutSplit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

In [4]:
X_full = pd.read_csv('New_Train_Features_Full.csv')
X_test_full = pd.read_csv('New_Test_Features_Full.csv')
y = pd.read_csv('Train_Target.csv').iloc[:,1]

X_full_no = pd.read_csv('New_Train_Features.csv')
X_test_full_no = pd.read_csv('New_Test_Features.csv')

X_no_manual = pd.read_csv('New_Train_Features_No_Selected.csv')
X_test_no_manual = pd.read_csv('New_Test_Features_No_Selected.csv')

X_RFE = pd.read_csv('New_Train_Features_RFE.csv')
X_RFE_test = pd.read_csv('New_Test_Features_RFE.csv')

X_LASSO = pd.read_csv('New_Train_Features_LASSO.csv')
X_LASSO_test = pd.read_csv('New_Test_Features_LASSO.csv')

X_manual = pd.read_csv('New_Train_Features_Selected.csv')
X_manual_test = pd.read_csv('New_Test_Features_Selected.csv')

# Full Dataset:

In [5]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [6]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [7]:
params = {
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "n_estimators": [100, 200, 300, 500],
    "criterion": ["friedman_mse", "squared_error"],
    "max_depth": [3, 5, 10, 20, None],
    "min_samples_split": [2, 5, 10],
    "max_features": [None, "sqrt", "log2"],
}

gb = GradientBoostingClassifier(random_state = 2025)

In [8]:
np.random.seed(2025)
gb_grid = GridSearchCV(gb, param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
gb_grid.fit(X_temp, y_temp)
print('Best Parameters:', gb_grid.best_params_)

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.05, 'max_depth': 5, 'max_features': None, 'min_samples_split': 10, 'n_estimators': 300}


In [9]:
np.random.seed(2025)
gb_model = GradientBoostingClassifier(criterion = 'friedman_mse', learning_rate = 0.05, max_depth = 5, max_features = None, min_samples_split = 10, n_estimators = 300, random_state = 2025)
gb_model.fit(X_temp, y_temp)
y_pred = gb_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [10]:
np.random.seed(2025)
gb_model.fit(X_full, y)
y_pred_gb = gb_model.predict(X_test_full)

In [11]:
from google.colab import files

pred_gb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_gb})

with open('GB_Full_Cov.csv', 'w') as file:
    pred_gb.to_csv(file, index = False, header = True)

files.download('GB_Full_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# RFE Dataset:

In [12]:
np.random.seed(2025)
X_train_rfe, X_temp_rfe, y_train_rfe, y_temp_rfe = train_test_split(X_RFE, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_rfe, X_test_rfe, y_val_rfe, y_test_rfe = train_test_split(X_temp_rfe, y_temp_rfe, test_size = 0.5, random_state = 2025, stratify = y_temp_rfe)

In [13]:
np.random.seed(2025)
train_rfe_ind, val_rfe_ind = train_test_split(np.arange(X_temp_rfe.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_rfe)
split_rfe = PredefinedHoldoutSplit(valid_indices = val_rfe_ind)

In [14]:
np.random.seed(2025)
gb_grid_rfe = GridSearchCV(GradientBoostingClassifier(random_state = 2025), param_grid = params, cv = split_rfe, n_jobs = -1, scoring = 'f1_macro')
gb_grid_rfe.fit(X_temp_rfe, y_temp_rfe)
print('Best Parameters:', gb_grid_rfe.best_params_)

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 100}


In [15]:
np.random.seed(2025)
gb_model_rfe = GradientBoostingClassifier(criterion = 'friedman_mse', learning_rate = 0.1, max_depth = 3, max_features = 'log2', min_samples_split = 10, n_estimators = 100, random_state = 2025)
gb_model_rfe.fit(X_temp_rfe, y_temp_rfe)
y_pred_rfe = gb_model_rfe.predict(X_test_rfe)

print(classification_report(y_test, y_pred_rfe))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [16]:
np.random.seed(2025)
gb_model_rfe.fit(X_RFE, y)
y_pred_rfe_gb = gb_model_rfe.predict(X_RFE_test)

In [17]:
from google.colab import files

pred_gb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_rfe_gb})

with open('GB_RFE_Cov.csv', 'w') as file:
    pred_gb.to_csv(file, index = False, header = True)

files.download('GB_RFE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# LASSO:

In [18]:
np.random.seed(2025)
X_train_lasso, X_temp_lasso, y_train_lasso, y_temp_lasso = train_test_split(X_LASSO, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val_lasso, X_test_lasso, y_val_lasso, y_test_lasso = train_test_split(X_temp_lasso, y_temp_lasso, test_size = 0.5, random_state = 2025, stratify = y_temp_lasso)

In [19]:
np.random.seed(2025)
train_lasso_ind, val_lasso_ind = train_test_split(np.arange(X_temp_lasso.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp_lasso)
split_lasso = PredefinedHoldoutSplit(valid_indices = val_lasso_ind)

In [20]:
np.random.seed(2025)
gb_grid_lasso = GridSearchCV(GradientBoostingClassifier(random_state = 2025), param_grid = params, cv = split_lasso, n_jobs = -1, scoring = 'f1_macro')
gb_grid_lasso.fit(X_temp_lasso, y_temp_lasso)
print('Best Parameters:', gb_grid_lasso.best_params_)

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'max_depth': 5, 'max_features': None, 'min_samples_split': 2, 'n_estimators': 500}


In [21]:
np.random.seed(2025)
gb_model_lasso = GradientBoostingClassifier(criterion = 'friedman_mse', learning_rate = 0.01, max_depth = 5, max_features = None, min_samples_split = 2, n_estimators = 500, random_state = 2025)
gb_model_lasso.fit(X_temp_lasso, y_temp_lasso)
y_pred_lasso = gb_model_lasso.predict(X_test_lasso)

print(classification_report(y_test, y_pred_lasso))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [22]:
np.random.seed(2025)
gb_model_lasso.fit(X_LASSO, y)
y_pred_lasso_gb = gb_model_lasso.predict(X_LASSO_test)

In [23]:
from google.colab import files

pred_gb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_lasso_gb})

with open('GB_LASSO_Cov.csv', 'w') as file:
    pred_gb.to_csv(file, index = False, header = True)

files.download('GB_LASSO_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization




In [24]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full_no, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [25]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [26]:
np.random.seed(2025)
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
gb_grid.fit(X_temp, y_temp)
print('Best Parameters:', gb_grid.best_params_)

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.05, 'max_depth': 5, 'max_features': None, 'min_samples_split': 10, 'n_estimators': 500}


In [27]:
np.random.seed(2025)
gb_model = GradientBoostingClassifier(criterion = 'friedman_mse', learning_rate = 0.05, max_depth = 5, max_features = None, min_samples_split = 10, n_estimators = 500, random_state = 2025)
gb_model.fit(X_temp, y_temp)
y_pred_gb_no = gb_model.predict(X_test)

print(classification_report(y_test, y_pred_gb_no))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [28]:
np.random.seed(2025)
gb_model.fit(X_full_no, y)
y_pred_gb_no = gb_model.predict(X_test_full_no)

In [29]:
from google.colab import files

pred_gb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_gb_no})

with open('GB_Full_No_Cov.csv', 'w') as file:
    pred_gb.to_csv(file, index = False, header = True)

files.download('GB_Full_No_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Full - Without Standardization - Manual



In [30]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_no_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [31]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [32]:
np.random.seed(2025)
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
gb_grid.fit(X_temp, y_temp)
print('Best Parameters:', gb_grid.best_params_)

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.05, 'max_depth': 5, 'max_features': None, 'min_samples_split': 10, 'n_estimators': 500}


In [33]:
np.random.seed(2025)
gb_model = GradientBoostingClassifier(criterion = 'friedman_mse', learning_rate = 0.05, max_depth = 5, max_features = None, min_samples_split = 10, n_estimators = 500, random_state = 2025)
gb_model.fit(X_temp, y_temp)
y_pred_manual = gb_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [34]:
np.random.seed(2025)
gb_model.fit(X_no_manual, y)
y_pred_gb_no_manual = gb_model.predict(X_test_no_manual)

In [35]:
from google.colab import files

pred_gb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_gb_no_manual})

with open('GB_No_Manual_Cov.csv', 'w') as file:
    pred_gb.to_csv(file, index = False, header = True)

files.download('GB_No_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Manual Selection:

In [36]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_manual, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

In [37]:
np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

In [38]:
np.random.seed(2025)
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
gb_grid.fit(X_temp, y_temp)
print('Best Parameters:', gb_grid.best_params_)

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.01, 'max_depth': 5, 'max_features': None, 'min_samples_split': 2, 'n_estimators': 500}


In [39]:
np.random.seed(2025)
gb_model = GradientBoostingClassifier(criterion = 'friedman_mse', learning_rate = 0.01, max_depth = 5, max_features = None, min_samples_split = 2, n_estimators = 500, random_state = 2025)
gb_model.fit(X_temp, y_temp)
y_pred_manual = gb_model.predict(X_test)

print(classification_report(y_test, y_pred_manual))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [40]:
np.random.seed(2025)
gb_model.fit(X_manual, y)
y_pred_gb_manual = gb_model.predict(X_manual_test)

In [41]:
from google.colab import files

pred_gb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_gb_manual})

with open('GB_Manual_Cov.csv', 'w') as file:
    pred_gb.to_csv(file, index = False, header = True)

files.download('GB_Manual_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# SMOTE FOR BEST GB:

In [42]:
from imblearn.over_sampling import SMOTE

In [45]:
np.random.seed(2025)
X_train, X_temp, y_train, y_temp = train_test_split(X_full, y, test_size = 0.3, random_state = 2025, stratify = y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 2025, stratify = y_temp)

np.random.seed(2025)
train_ind, val_ind = train_test_split(np.arange(X_temp.shape[0]), test_size = 0.3, random_state = 2025, stratify = y_temp)
split = PredefinedHoldoutSplit(valid_indices = val_ind)

smote = SMOTE(random_state = 2025)
X_resampled, y_resampled = smote.fit_resample(X_temp, y_temp)

In [46]:
np.random.seed(2025)
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state = 2025), param_grid = params, cv = split, n_jobs = -1, scoring = 'f1_macro')
gb_grid.fit(X_temp, y_temp)
print('Best Parameters:', gb_grid.best_params_)

Best Parameters: {'criterion': 'friedman_mse', 'learning_rate': 0.05, 'max_depth': 5, 'max_features': None, 'min_samples_split': 10, 'n_estimators': 300}


In [47]:
np.random.seed(2025)
gb_model_smote = GradientBoostingClassifier(criterion = 'friedman_mse', learning_rate = 0.05, max_depth = 5, max_features = None, min_samples_split = 10, n_estimators = 300, random_state = 2025)
gb_model_smote.fit(X_resampled, y_resampled)
y_pred_smote = gb_model_smote.predict(X_test)

print(classification_report(y_test, y_pred_smote))
y_pred_smote_gb = gb_model_smote.predict(X_test_full)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00         9

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [48]:
from google.colab import files

pred_gb = pd.DataFrame({'ID': range(1, 92), 'Placement': y_pred_smote_gb})

with open('GB_SMOTE_Cov.csv', 'w') as file:
    pred_gb.to_csv(file, index = False, header = True)

files.download('GB_SMOTE_Cov.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>